## Deploying DOcplex and control parameters with Watson Machine Learning

This notebook shows you how to deploy a DOcplex model and pass environment variables to control the flow control in our Decision Optimization Python script using the Watson Machine Learning Python Client.

This notebook runs on Python.

**Table of contents:**

- [Set up the Watson Machine Learning client](#setup)
- [Create a client instance](#create)
- [Prepare your model archive](#prepare)
- [Upload your model on Watson Machine Learning](#upload)
- [Create a deployment](#deploy)
- [Create and monitor a job for your deployed model](#job)
- [Summary](#summary)

<a id='setup'></a>
### Set up the Watson Machine Learning client

Install and then import the Watson Machine Learning client library.

In [ ]:
# Install WML client API

!pip install ibm-watson-machine-learning

In [ ]:
from ibm_watson_machine_learning import APIClient

<a id='create'></a>
### Create a client instance

In [ ]:
# Instantiate a client using credentials

cluster = "<your_cluster>"
username = "<username>"
password = "<password>"

wml_credentials = {
    "username": username,
    "password": password,
    "instance_id" : "wml_local",
    "url": cluster,
    "version": "4.8"
}

client = APIClient(wml_credentials)

In [ ]:
client.version

<a id='prepare'></a>
### Prepare your model archive

Use the `write_file` command to write these models to a `model.py` file.

This model solves this geometric puzzle:  
Start with a pattern of circles placed in rows piled one on top of the other with decreasing numbers in each row to form a triangle. With N circles at the bottom, then N-1 circles in the adjacent row, then N-2 in the next row etc ... until there is just 1 circle in the top row, you must decide which circles to color in, so that the maximum number of circles are colored in. There is however the constraint that no 3 selected circles form a triangle. Hence the center of any circle is considered as a vertex for a potential triangle.

Use the `tar` command to create a tar archive.

In [ ]:
%mkdir model

In [ ]:
%%writefile model.py

from docplex.mp.model import Model

def build_hearts(r, **kwargs):
    # initialize the model
    mdl = Model('love_hearts_%d' % r, **kwargs)

    # the dictionary of decision variables, one variable
    # for each circle with i in (1 .. r) as the row and
    # j in (1 .. i) as the position within the row    
    idx = [(i, j) for i in range(1, r + 1) for j in range(1, i + 1)]
    a = mdl.binary_var_dict(idx, name=lambda ij: "a_%d_%d" % ij)

    # the constraints - enumerate all equilateral triangles
    # and prevent any such triangles from being chosen by keeping
    # the number of chosen circles with adjacent vertices below 3

    # for each row except the last
    for i in range(1, r):
        # for each position in this row
        for j in range(1, i + 1):
            # for each triangle of side length (k) with its upper vertex at
            # (i, j) and its sides parallel to those of the overall shape
            for k in range(1, r - i + 1):
                # the sets of 3 points at the same distances clockwise along the
                # sides of these triangles form k equilateral triangles
                for m in range(k):
                    u, v, w = (i + m, j), (i + k, j + m), (i + k - m, j + k - m)
                    mdl.add(a[u] + a[v] + a[w] <= 2)

    mdl.maximize(mdl.sum(a))
    return mdl

from docplex.util.environment import get_environment

env = get_environment()
debug = env.get_parameter("MY_DEBUG_FLAG")
if debug is not None:
    print("DEBUG MODE ON")
    print(debug)
else:
    print("DEBUG MODE OFF")
       

mdl = build_hearts(5)
mdl.solve(log_output=True if debug is not None else False)

In [ ]:
import tarfile
def reset(tarinfo):
    tarinfo.uid = tarinfo.gid = 0
    tarinfo.uname = tarinfo.gname = "root"
    return tarinfo
tar = tarfile.open("model.tar.gz", "w:gz")
tar.add("model.py", arcname="model.py", filter=reset)
tar.close()


<a id='upload'></a>
### Upload your model on Watson Machine Learning

Store model in Watson Machine Learning with:
* the tar archive previously created,
* metadata including the model type and runtime

Get the `model_uid`.

In [ ]:
# Find the space ID

space_name = "<space_name>"

space_id = [x['metadata']['id'] for x in client.spaces.get_details()['resources'] if x['entity']['name'] == space_name][0]

client.set.default_space(space_id)

In [ ]:
mnist_metadata = {
    client.repository.ModelMetaNames.NAME: "MyModel",
    client.repository.ModelMetaNames.DESCRIPTION: "Model for Loving Hearts",
    client.repository.ModelMetaNames.TYPE: "do-docplex_22.1",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: client.software_specifications.get_uid_by_name("do_22.1"),
}

model_details = client.repository.store_model(model='/home/wsuser/work/model.tar.gz', meta_props=mnist_metadata)
#model='/home/wsuser/work/model.tar.gz', 
model_uid = client.repository.get_model_id(model_details)

<a id='deploy'></a>
### Create a deployment 

Create a batch deployment for the model, providing information such as:
* the maximum number of compute nodes
* the T-shirt size of the compute nodes

Get the `deployment_uid`.

In [ ]:
meta_props = {
    client.deployments.ConfigurationMetaNames.NAME: "Loving Hearts Deployment",
    client.deployments.ConfigurationMetaNames.DESCRIPTION: "Loving Hearts Deployment",
    client.deployments.ConfigurationMetaNames.BATCH: {},
    client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: {'name': 'S', 'num_nodes': 1}
}

deployment_details = client.deployments.create(model_uid, meta_props=meta_props)

deployment_uid = client.deployments.get_uid(deployment_details)

# print deployment id if needed
# print( deployment_uid )

In [ ]:
# List all existing deployments

client.deployments.list()

<a id='job'></a>
### Create and monitor a job  for your deployed model

Create a payload containing inline input data.

Create a new job with this payload and the deployment.
No specific parameter.

Get the `job_uid`.

In [ ]:
solve_payload = {
    "solve_parameters" : {
        "oaas.logAttachmentName":"log.txt",
        "oaas.logTailEnabled":"true",
        "oaas.resultsFormat": "XML"
    },
    client.deployments.DecisionOptimizationMetaNames.INPUT_DATA: [
    ],
    client.deployments.DecisionOptimizationMetaNames.OUTPUT_DATA: [
        {
            "id":".*\.xml"
        },
        {
            "id":"log.txt"
        }
    ]
}
job_details = client.deployments.create_job(deployment_uid, solve_payload)
job_uid = client.deployments.get_job_uid(job_details)

Display job status until it is completed.

The first job of a new deployment might take some time as a compute node must be started.

In [ ]:
from time import sleep

while job_details['entity']['decision_optimization']['status']['state'] not in ['completed', 'failed', 'canceled']:
    print(job_details['entity']['decision_optimization']['status']['state'] + '...')
    sleep(5)
    job_details=client.deployments.get_job_details(job_uid)

print(job_details['entity']['decision_optimization']['solve_state']['solve_status'])

Print the logs: no parameter is passed to DOcplex so the model is not in debug mode.

In [ ]:
import base64
import io

output_data = job_details['entity']['decision_optimization']['output_data']

logs = [line for o in output_data if o['id'] == 'log.txt' for line in io.BytesIO(base64.b64decode(o['content']))]
for l in logs:
    print(l)

In [ ]:
check_log = [l for l in logs if "DEBUG MODE OFF" in str(l)]
if len(check_log) == 1:
    print("Solve was ok")
else:
    print("Something went wrong")
print(check_log)

##### Run the model and pass it optimization control parameters

In [ ]:
solve_payload = {
    "solve_parameters" : {
        "oaas.logAttachmentName":"log.txt",
        "oaas.logTailEnabled":"true",
        "oaas.resultsFormat": "XML",
        "MY_DEBUG_FLAG" : "my debug flag value"
    },
    client.deployments.DecisionOptimizationMetaNames.INPUT_DATA: [
    ],
    client.deployments.DecisionOptimizationMetaNames.OUTPUT_DATA: [
        {
            "id":".*\.xml"
        },
        {
            "id":"log.txt"
        }
    ]
}
job_details = client.deployments.create_job(deployment_uid, solve_payload)
job_uid = client.deployments.get_job_uid(job_details)

In [ ]:
from time import sleep

while job_details['entity']['decision_optimization']['status']['state'] not in ['completed', 'failed', 'canceled']:
    print(job_details['entity']['decision_optimization']['status']['state'] + '...')
    sleep(5)
    job_details=client.deployments.get_job_details(job_uid)

print(job_details['entity']['decision_optimization']['solve_state']['solve_status'])

The optimization parameters were passed so DOcplex will be in debug mode.

In [ ]:
import base64
import io

output_data = job_details['entity']['decision_optimization']['output_data']

logs = [line for o in output_data if o['id'] == 'log.txt' for line in io.BytesIO(base64.b64decode(o['content']))]
for l in logs:
    print(l)

In [ ]:
check_log = [l for l in logs if "DEBUG MODE ON" in str(l)]
if len(check_log) == 1:
    print("Solve was ok")
else:
    print("Something went wrong")
print(check_log)

### Delete the deployment

Use the following method to delete the deployment.

In [ ]:
client.deployments.delete(deployment_uid)

<a id='summary'></a>
### Summary and next steps

You've successfully completed this notebook! 

You've learned how to:

- work with the Watson Machine Learning client
- prepare your model archive and upload your model on Watson Machine Learning
- create a deployment
- create and monitor a job with inline data for your deployed model

Check out our online documentation <a href="https://www.ibm.com/docs/en/cloud-paks/cp-data/4.8.x" target="_blank" rel="noopener noreferrer">here</a> for more samples, tutorials and documentation.

<hr>
Copyright © 2019-2024. This notebook and its source code are released under the terms of the MIT License.